In [2]:
import cv2
import numpy as np
from keras_facenet import FaceNet
import face_recognition
from scipy.spatial.distance import cosine

# Initialize FaceNet

In [3]:
embedder = FaceNet()

def get_face_embedding(image):
    face_locations = face_recognition.face_locations(image)
    if len(face_locations) == 0:
        return None, None

    # Take first face
    top, right, bottom, left = face_locations[0]
    face = image[top:bottom, left:right]
    face_resized = cv2.resize(face, (160, 160))
    face_rgb = cv2.cvtColor(face_resized, cv2.COLOR_BGR2RGB)
    embedding = embedder.embeddings([face_rgb])[0]
    return embedding, (top, right, bottom, left)

def compare_embeddings(embedding1, embedding2, threshold=0.5):
    distance = cosine(embedding1, embedding2)
    return distance < threshold, distance


# Load and embed the reference face

In [4]:
reference_image = cv2.imread("reference_person.jpg")
ref_embedding, _ = get_face_embedding(reference_image)
if ref_embedding is None:
    print("❌ No face found in reference image. Exiting.")
    exit()


TypeError: __call__(): incompatible function arguments. The following argument types are supported:
    1. (self: _dlib_pybind11.fhog_object_detector, image: numpy.ndarray, upsample_num_times: int = 0) -> _dlib_pybind11.rectangles

Invoked with: <_dlib_pybind11.fhog_object_detector object at 0x000001BEBB4CCD70>, None, 1

# Start webcam

In [ ]:
cap = cv2.VideoCapture(0)
print("🚀 Starting webcam. Press 'q' to exit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    embedding, face_coords = get_face_embedding(frame)

    if embedding is not None:
        match, distance = compare_embeddings(ref_embedding, embedding)
        label = f"Match ({distance:.2f})" if match else f"No Match ({distance:.2f})"
        color = (0, 255, 0) if match else (0, 0, 255)
        top, right, bottom, left = face_coords
        cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
        cv2.putText(frame, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
    else:
        cv2.putText(frame, "No Face Detected", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    cv2.imshow("Live Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()